tgb - 11/13/2019 - Testing if rescaling the inputs changes the ability of the network to generalize.   
Architecture = From [], predict []   
Test 1 = Rephrase all water concentrations as RH  
Test 2 = Rephrase all temperature as deviations from moist adiabat  
  
tgb - 11/15/2019 - This notebook is mostly used for pre-processing

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog
[Errno 2] No such file or directory: '/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM'
/nfspool-0/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog


# 1) Preprocessing

In [2]:
%cd /export/nfs0home/tbeucler/CBRAIN-CAM

/nfspool-0/home/tbeucler/CBRAIN-CAM


In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_06_06_RG_Test-0K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/06/2021 10:28:44 AM Preprocess training dataset with real_geography flag set to True
06/06/2021 10:28:44 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_06_RG_TEST.nc
06/06/2021 10:28:44 AM Reading input files
06/06/2021 10:28:44 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_0001Kp_Neuralnet_SPCAM_all_216/atm/hist/SST_0001Kp_Neuralnet_SPCAM_all_216.cam.h1.2014-*-2*-*.nc
06/06/2021 10:37:38 AM Crop levels
06/06/2021 10:37:38 AM Create stacked dataarray
06/06/2021 10:37:38 AM Real geography flag set to True
06/06/2021 10:37:38 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINIT

In [3]:
!python preprocessing-11132019.py -c pp_config/2021_06_03_RG_Train_Valid-0K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
06/04/2021 07:55:18 AM Preprocess training dataset with real_geography flag set to True
06/04/2021 07:55:18 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_06_03_RG_TRAIN.nc
06/04/2021 07:55:18 AM Reading input files
06/04/2021 07:55:18 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_0001Kp_Neuralnet_SPCAM_all_216/atm/hist/SST_0001Kp_Neuralnet_SPCAM_all_216.cam.h1.2013-*-0*-*.nc
06/04/2021 08:03:40 AM Crop levels
06/04/2021 08:03:40 AM Create stacked dataarray
06/04/2021 08:03:40 AM Real geography flag set to True
06/04/2021 08:03:40 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX
var is  LHFLX
var is  PHQ
var is  TPHYSTND
var is  QRL
var is  QRS
OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0
OMP: Info #156: KMP_AFFINI

In [18]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 01:07:56 AM Preprocess training dataset including output quantiles
04/25/2021 01:07:56 AM with real_geography flag set to True
04/25/2021 01:07:56 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TRAIN_M4K.nc
04/25/2021 01:07:56 AM Reading input files
04/25/2021 01:07:56 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2009-*-0*-*.nc
04/25/2021 01:12:31 AM Reading PKL file containing univariate distribution of output variables
04/25/2021 01:12:31 AM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TRAIN_M4K_shuffle.nc_PERC.pkl
04/25/2021 01:12:31 AM Crop levels
04/25/2021 01:12:31 AM Create stacked dataarray
04/25/2021 01:12:31 AM Real geography flag set to True
04/25/2021 01:12:31 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLI

OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128624 thread 24 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128626 thread 25 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128589 thread 26 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128598 thread 27 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128616 thread 28 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128617 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128584 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128592 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128591 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128601 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128599 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 126111 tid 128611 thread 35 bou

04/25/2021 08:10:34 AM Done!
04/25/2021 08:10:42 AM Finish entire preprocessing script.


In [19]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Train_Valid-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 08:16:12 AM Preprocess training dataset including output quantiles
04/25/2021 08:16:12 AM with real_geography flag set to True
04/25/2021 08:16:12 AM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TRAIN_P4K.nc
04/25/2021 08:16:12 AM Reading input files
04/25/2021 08:16:12 AM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2009-*-0*-*.nc
04/25/2021 08:21:25 AM Reading PKL file containing univariate distribution of output variables
04/25/2021 08:21:25 AM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TRAIN_P4K_shuffle.nc_PERC.pkl
04/25/2021 08:21:25 AM Crop levels
04/25/2021 08:21:25 AM Create stacked dataarray
04/25/2021 08:21:25 AM Real geography flag set to True
04/25/2021 08:21:25 AM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var 

04/25/2021 04:21:17 PM Done!
04/25/2021 04:21:22 PM Finish entire preprocessing script.


In [20]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-P4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/25/2021 04:26:51 PM Preprocess training dataset including output quantiles
04/25/2021 04:26:51 PM with real_geography flag set to True
04/25/2021 04:26:51 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TEST_P4K.nc
04/25/2021 04:26:51 PM Reading input files
04/25/2021 04:26:51 PM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Kp_Neuralnet_SPCAM_all_v2_216/atm/hist/SST_4Kp_Neuralnet_SPCAM_all_v2_216.cam.h1.2010-*-2*-*.nc
04/25/2021 04:32:30 PM Reading PKL file containing univariate distribution of output variables
04/25/2021 04:32:30 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TEST_P4K.nc_PERC.pkl
04/25/2021 04:32:30 PM Crop levels
04/25/2021 04:32:30 PM Create stacked dataarray
04/25/2021 04:32:30 PM Real geography flag set to True
04/25/2021 04:32:30 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  SHFLX


OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101687 thread 24 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101683 thread 25 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101660 thread 26 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101693 thread 27 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101662 thread 28 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101686 thread 29 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101653 thread 30 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101685 thread 31 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101690 thread 32 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101692 thread 33 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101672 thread 34 bound to OS proc set 0
OMP: Info #250: KMP_AFFINITY: pid 95391 tid 101655 thread 35 bound to OS pro

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_24_RG_PERC_Test-M4K.yml

/nfspool-0/home/tbeucler/CBRAIN-CAM
04/26/2021 04:16:59 PM Preprocess training dataset including output quantiles
04/26/2021 04:16:59 PM with real_geography flag set to True
04/26/2021 04:16:59 PM Start preprocessing file /DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/2021_04_24_RG_PERC_TEST_M4K.nc
04/26/2021 04:16:59 PM Reading input files
04/26/2021 04:16:59 PM Reading input file /DFS-L/DATA/pritchard/liranp/SST_4Km_Neuralnet_SPCAM_all_small_216/atm/hist/SST_4Km_Neuralnet_SPCAM_all_small_216.cam.h1.2010-*-2*-*.nc
04/26/2021 04:21:47 PM Reading PKL file containing univariate distribution of output variables
04/26/2021 04:21:47 PM Reading /export/home/tbeucler/CBRAIN-CAM/notebooks/tbeucler_devlog/PKL_DATA/2021_04_18_RG_TEST_M4K.nc_PERC.pkl
04/26/2021 04:21:47 PM Crop levels
04/26/2021 04:21:47 PM Create stacked dataarray
04/26/2021 04:21:47 PM Real geography flag set to True
04/26/2021 04:21:47 PM Earth-like simulation detected
var is  QBP
var is  TBP
var is  PS
var is  SOLIN
var is  

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_04_09_PERC_Test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_31_O3_T_NSto220_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-P4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Train_Valid-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_18_O3_Test-M4K.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_02_01_O3_BCONS_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsDELQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_03_01_O3_LHFnsQ_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_02_01_O3_QSATdeficit_input_small_data_test.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_input_small_data_test

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Train_Valid.yml

In [ ]:
!python preprocessing-11132019.py -c /export/nfs0home/tbeucler/CBRAIN-CAM/pp_config/2021_01_24_O3_Test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/147_POG_RHinput_FLUXoutput_test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/118_POG_TfromTSinput_test.yml

In [ ]:
!ls -la -h /local/Tom.Beucler/SPCAM_PHYS/1*

# 2) Check the preprocessed data

In [11]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [12]:
path_RGsmall = path_data + '2021_04_18_NORM_RG_small.nc'

In [ ]:
path_QSATdeficit = path_data+'2021_02_01_NORM_O3_QSATdeficit_small.nc'

In [ ]:
path_TfromNS = path_data+'2021_02_01_NORM_O3_TfromNS_small.nc'

In [ ]:
path_Bcons = path_data+'2021_03_17_NORM_O3_BCONS_small.nc'

In [13]:
data_RGsmall = xr.open_dataset(path_RGsmall)

In [ ]:
data_QSATdeficit = xr.open_dataset(path_QSATdeficit)
data_TfromNS = xr.open_dataset(path_TfromNS)
data_Bcons = xr.open_dataset(path_Bcons)

In [ ]:
data_QSATdeficit

In [ ]:
data_TfromNS

In [15]:
data_RGsmall['var_names']

<xarray.DataArray 'var_names' (var_names: 184)>
array(['QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP', 'QBP',
       'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP',
       'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP',
       'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP', 'TBP',
       'PS', 'SOLIN', 'SHFLX', 'LHFLX', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ',
       'PHQ', 'PHQ', 'PHQ', 'PHQ', 'PHQ', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND', 'TPHYSTND',
       'TPHYSTND', 'TPHYS

In [ ]:
data_TfromNS['mean'].shape

In [ ]:
data_QSATdeficit['mean'][:30]

In [ ]:
data_TfromNS['mean'][30:60]

In [ ]:
data_Bcons['mean'][30:60]

In [ ]:
data_Bcons['std'][30:60]

# 3) Preprocessing for PRL paper

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-RH-test.yml

In [ ]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/8col_rad_tbeucler_local-test.yml